In [1]:
import gym
from gym import wrappers
import torch
import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F
from collections import namedtuple
from torch.autograd import Variable
import random
import numpy as np
from collections import deque

In [2]:
env = gym.make('CartPole-v0')
env = wrappers.Monitor(env, 'cartpole', force=True)

[2017-11-27 23:03:49,908] Making new env: CartPole-v0
[2017-11-27 23:03:49,933] Clearing 4 monitor files from previous run (because force=True was provided)


In [3]:
class Policy(nn.Module):
  def __init__(self, input_size, output_size):
    super(Policy, self).__init__() # this statement is always needed
    
    self.fc1 = nn.Linear(input_size, 10) # matrix multiplication
    self.fc2 = nn.Linear(10, output_size) # matrix multiplication
    
    # == parameters initialization ==
    nn.init.xavier_normal(self.fc1.weight)
    nn.init.xavier_normal(self.fc2.weight)
    
    nn.init.normal(self.fc1.bias)
    nn.init.normal(self.fc2.bias)
    # =============================== 
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [4]:
class ValueFunction(nn.Module):
  def __init__(self, input_size):
    super(ValueFunction, self).__init__()
    
    self.fc1 = nn.Linear(input_size, 10)
    self.fc2 = nn.Linear(10, 1) # a single value
    
    # == parameters initialization ==
    nn.init.xavier_normal(self.fc1.weight)
    nn.init.xavier_normal(self.fc2.weight)
    
    nn.init.normal(self.fc1.bias)
    nn.init.normal(self.fc2.bias)
    # ===============================
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [5]:
class ActorCritic(nn.Module):
  def __init__(self, input_size, output_size):
    super(ActorCritic, self).__init__()
    self.actor = Policy(input_size, output_size)
    self.critic = ValueFunction(input_size)

  def forward(self, x):
    return self.critic(x), self.actor(x) # value, action

In [6]:
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
print('input_size = {0}, output_size = {1}'.format(input_size, output_size))

input_size = 4, output_size = 2


In [7]:
gamma = 0.99 # the parameter for discounting future rewards
lbda = 0.9 # the parameter for GAE (generalized advantage estimation)

In [8]:
model = ActorCritic(input_size, output_size)
optimizer = torch.optim.Adam(model.parameters())

In [9]:
# vanilla policy gradient
for i in range(2000):
  current_state = env.reset() # an array of 4 values
  done = False
  episode_reward = 0
  
  values = [] 
  logprobs = []
  rewards = []

  while not done:
     # forward propagation on policy using current_state
    value, action_real = model(Variable(torch.from_numpy(current_state).float().unsqueeze(0))) 
    # value and action_real are variables
    action_logprob = F.log_softmax(action_real) # returns a variable
    
    action_prob = F.softmax(action_real) # returns a variable
    action = action_prob.multinomial().data # returns a torch tensor
    
    logprob = action_logprob.gather(1, Variable(action))
    current_state, reward, done, _ = env.step(action.numpy()[0,0])
    
    values.append(value) # variable
    logprobs.append(logprob) # variable
    rewards.append(reward) # numpy
    
    episode_reward += reward
  
  R = 0
  value_loss = 0
  policy_loss = 0
  for j in reversed(range(len(rewards))):
    R = rewards[j] + gamma * R # numpy
    advantage = R - values[j]
    value_loss += advantage.pow(2)
    policy_loss -= logprobs[j] * advantage
  
  optimizer.zero_grad()
  value_loss.backward(retain_variables=True)
  policy_loss.backward()
  optimizer.step()
  
  print('{0}: episode_rewards = {1}, policy_loss = {2}, value_loss = {3}'.format(
    i, episode_reward, policy_loss.data[0,0], value_loss.data[0,0]))

[2017-11-27 23:04:08,315] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.68446.video000000.mp4
/Users/fchua/Applications/miniconda/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "
[2017-11-27 23:04:09,408] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.68446.video000001.mp4


0: episode_rewards = 11.0, policy_loss = 44.216556549072266, value_loss = 650.34375


/Users/fchua/Applications/miniconda/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "
[2017-11-27 23:04:10,053] Starting new video recorder writing to /Users/fchua/Documents/torch_projects/pytorch_tutorials/pytorch-deep-rl/cartpole/openaigym.video.0.68446.video000008.mp4


1: episode_rewards = 28.0, policy_loss = 298.5724182128906, value_loss = 7316.267578125
2: episode_rewards = 21.0, policy_loss = 164.59884643554688, value_loss = 3428.46826171875
3: episode_rewards = 13.0, policy_loss = 62.25950241088867, value_loss = 991.8350219726562
4: episode_rewards = 12.0, policy_loss = 54.986114501953125, value_loss = 808.2779541015625
5: episode_rewards = 12.0, policy_loss = 68.74180603027344, value_loss = 777.2911987304688
6: episode_rewards = 11.0, policy_loss = 60.19776153564453, value_loss = 621.7948608398438
7: episode_rewards = 16.0, policy_loss = 98.62602996826172, value_loss = 1687.5443115234375


/Users/fchua/Applications/miniconda/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


...

1560: episode_rewards = 197.0, policy_loss = 4263.71923828125, value_loss = 367148.5625
1561: episode_rewards = 200.0, policy_loss = 4002.35400390625, value_loss = 358555.71875
1562: episode_rewards = 45.0, policy_loss = -63.61740493774414, value_loss = 5282.55908203125
1563: episode_rewards = 200.0, policy_loss = 4533.7109375, value_loss = 372876.53125
1564: episode_rewards = 200.0, policy_loss = 4206.65576171875, value_loss = 349996.25
1565: episode_rewards = 200.0, policy_loss = 4320.39990234375, value_loss = 377340.8125
1566: episode_rewards = 200.0, policy_loss = 4178.75, value_loss = 339471.5625
1567: episode_rewards = 183.0, policy_loss = 3637.634033203125, value_loss = 306254.0
1568: episode_rewards = 200.0, policy_loss = 3922.526123046875, value_loss = 348733.71875
1569: episode_rewards = 105.0, policy_loss = 825.556884765625, value_loss = 78585.8125
1570: episode_rewards = 200.0, policy_loss = 4019.359375, value_loss = 351510.21875
1571: episode_rewards = 200.0, policy_loss =

1653: episode_rewards = 179.0, policy_loss = 3018.573486328125, value_loss = 251889.0625
1654: episode_rewards = 129.0, policy_loss = 1457.8017578125, value_loss = 103032.375
1655: episode_rewards = 133.0, policy_loss = 1636.1781005859375, value_loss = 106409.0625
1656: episode_rewards = 200.0, policy_loss = 3724.837158203125, value_loss = 321337.3125
1657: episode_rewards = 200.0, policy_loss = 3904.8544921875, value_loss = 324929.84375
1658: episode_rewards = 117.0, policy_loss = 1112.1968994140625, value_loss = 75732.625
1659: episode_rewards = 200.0, policy_loss = 3847.247802734375, value_loss = 320986.6875
1660: episode_rewards = 200.0, policy_loss = 3830.60302734375, value_loss = 291309.15625
1661: episode_rewards = 124.0, policy_loss = 1309.5904541015625, value_loss = 92160.5703125
1662: episode_rewards = 200.0, policy_loss = 3477.943603515625, value_loss = 287137.71875
1663: episode_rewards = 160.0, policy_loss = 2280.896728515625, value_loss = 189788.609375
1664: episode_rewar

1747: episode_rewards = 200.0, policy_loss = 3057.15087890625, value_loss = 267577.71875
1748: episode_rewards = 131.0, policy_loss = 957.1151123046875, value_loss = 84220.3203125
1749: episode_rewards = 170.0, policy_loss = 2406.108642578125, value_loss = 184939.359375
1750: episode_rewards = 200.0, policy_loss = 2918.9091796875, value_loss = 269667.34375
1751: episode_rewards = 66.0, policy_loss = -252.48184204101562, value_loss = 24140.52734375
1752: episode_rewards = 200.0, policy_loss = 3193.734375, value_loss = 281418.0625
1753: episode_rewards = 174.0, policy_loss = 2507.9892578125, value_loss = 199258.109375
1754: episode_rewards = 200.0, policy_loss = 2877.37548828125, value_loss = 257013.875
1755: episode_rewards = 200.0, policy_loss = 2658.893310546875, value_loss = 273701.59375
1756: episode_rewards = 200.0, policy_loss = 3463.37939453125, value_loss = 272518.1875
1757: episode_rewards = 200.0, policy_loss = 3121.762451171875, value_loss = 264457.375
1758: episode_rewards =

1840: episode_rewards = 200.0, policy_loss = 2930.0869140625, value_loss = 231313.75
1841: episode_rewards = 161.0, policy_loss = 1870.2764892578125, value_loss = 130757.875
1842: episode_rewards = 200.0, policy_loss = 2503.6259765625, value_loss = 209938.421875
1843: episode_rewards = 146.0, policy_loss = 1299.4935302734375, value_loss = 105215.8828125
1844: episode_rewards = 200.0, policy_loss = 2846.998779296875, value_loss = 224342.828125
1845: episode_rewards = 200.0, policy_loss = 2796.6220703125, value_loss = 238889.265625
1846: episode_rewards = 200.0, policy_loss = 2540.490478515625, value_loss = 225995.390625
1847: episode_rewards = 200.0, policy_loss = 2463.93359375, value_loss = 218327.390625
1848: episode_rewards = 164.0, policy_loss = 1830.203125, value_loss = 142117.171875
1849: episode_rewards = 164.0, policy_loss = 1832.0919189453125, value_loss = 140025.125
1850: episode_rewards = 200.0, policy_loss = 2622.5205078125, value_loss = 214697.234375
1851: episode_rewards =

1932: episode_rewards = 174.0, policy_loss = 1710.78173828125, value_loss = 142564.578125
1933: episode_rewards = 200.0, policy_loss = 2364.378662109375, value_loss = 200869.53125
1934: episode_rewards = 184.0, policy_loss = 1411.1220703125, value_loss = 166932.46875
1935: episode_rewards = 200.0, policy_loss = 2254.45166015625, value_loss = 196470.703125
1936: episode_rewards = 128.0, policy_loss = 526.2305908203125, value_loss = 60828.68359375
1937: episode_rewards = 196.0, policy_loss = 1792.469970703125, value_loss = 189780.1875
1938: episode_rewards = 195.0, policy_loss = 2387.105712890625, value_loss = 187296.4375
1939: episode_rewards = 200.0, policy_loss = 2091.43701171875, value_loss = 183030.921875
1940: episode_rewards = 46.0, policy_loss = -583.773193359375, value_loss = 19446.439453125
1941: episode_rewards = 200.0, policy_loss = 2370.7001953125, value_loss = 180984.84375
1942: episode_rewards = 200.0, policy_loss = 2176.4560546875, value_loss = 186872.015625
1943: episode

In [ ]:
env.render(close=True)
env.close()

In [ ]:
env = gym.make('Pendulum-v0')

In [ ]:
input_size = env.observation_space.shape[0]
output_size = env.action_space.shape[0]
print('input_size = {0}, output_size = {1}'.format(input_size, output_size))

In [ ]:
model = ActorCritic(input_size, output_size)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# vanilla policy gradient
for i in range(2000):
  current_state = env.reset() # an array of 4 values
  done = False
  episode_reward = 0
  
  values = []
  logprobs = []
  rewards = []

  while not done:
     # forward propagation on policy using current_state
    value, action_real = model(Variable(torch.from_numpy(current_state).float().unsqueeze(0)))
    break
    # value and action_real are variables
#     action_logprob = F.log_softmax(action_real) # returns a variable
    
#     action_prob = F.softmax(action_real) # returns a variable
#     action = action_prob.multinomial().data # returns a torch tensor
    
#     logprob = action_logprob.gather(1, Variable(action))
#     current_state, reward, done, _ = env.step(action.numpy()[0,0])
    
#     values.append(value) # variable
#     logprobs.append(logprob) # variable
#     rewards.append(reward) # numpy
    
#     episode_reward += reward
  
#   R = 0
#   value_loss = 0
#   policy_loss = 0
#   for j in reversed(range(len(rewards))):
#     R = rewards[j] + gamma * R # numpy
#     advantage = R - values[j]
#     value_loss += advantage.pow(2)
#     policy_loss -= logprobs[j] * advantage
  
#   optimizer.zero_grad()
#   value_loss.backward(retain_variables=True)
#   policy_loss.backward()
#   optimizer.step()
  
#   print('{0}: episode_rewards = {1}, policy_loss = {2}, value_loss = {3}'.format(
#     i, episode_reward, policy_loss.data[0,0], value_loss.data[0,0]))

In [ ]:
F.tanh(action_real)